In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from external.pydominion.dominion import Piles, Phase
from external.pydominion.dominion.Game import Game

## For any testing

In [ ]:
game = Game(
    numplayers=2,
    initcards=["Cellar", "Market", "Militia", "Mine", "Moat", 
                "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
    validate_only=False,
    prosperity=False,
    potions = False,
    shelters = False,
    card_path="external/pydominion/dominion/cards"
)

In [ ]:
game.start_game()

In [ ]:
# Testing to look into the options a bit more
options = game.current_player._choice_selection()
options[0]

# Registration/Creation/Playing
This all works right now

In [ ]:
import gymnasium as gym

In [ ]:
#Register
gym.register(
    id="Dominion-v14",
    entry_point="DominionEnv:DominionEnv"
)

In [ ]:
env = gym.make("Dominion-v14", 
         num_players=2, 
         card_set=["Cellar", "Market", "Militia", "Mine", "Moat", 
                   "Remodel", "Smithy", "Village", "Throne Room", "Workshop"],
         quiet_flag=True
        )


In [ ]:
obs, _ = env.reset()

The code below is what I was using to test when it was still player input and not an agent learning. Don't run it

In [ ]:
# Set up some basic parameters for the loop
episode_over = False
total_reward = 0
step_count = 0

# Run the game loop
while not episode_over and step_count < 100:  # Limit to 100 steps to prevent infinite loops
    step_count += 1

    action = env.action_space.sample() 
    observation, reward, terminated, truncated, info = env.step(action)

    episode_over = terminated or truncated

# Print end game
print(f"Game finished! Total reward: {total_reward}")

# Test with an agent?
Works... but not really because agent can handle user input I don't think

In [ ]:
# This runs in a minute, do we need to update our timesteps <-- Look through this more closely!
from stable_baselines3 import PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")
model = PPO.load("ppo_cartpole")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 703  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 515         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013879124 |
|    clip_fraction        | 0.156       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.76       |
|    explained_variance   | -7.185498   |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00606    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0122     |
|    value_loss         

In [ ]:
# This takes longer then 45 minutes to run try and figure out what is happening!
obs, _ = env.reset()
while True:
    action, _ = model.predict(obs)
    observation, reward, terminated, truncated, info = env.step(action)